In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import scipy.integrate
import scipy.sparse as sp

In [ ]:
#constants
hbar, m = 1, 1
k0, L = 20, 20
E = (hbar * k0) ** 2 / 2 / m
omega = E / hbar


def psi_I_anal(x, t):
    return np.exp(-(x - x0)**2 / (2 * sigmax**2)) * np.sin(k0 * x - omega * t)
                  
def psi_R_anal(x,t):
    return np.exp(-(x - x0)**2 / (2 * sigmax **2)) * np.cos(k0 * x - omega * t)


def make_psi_init(x0,sigmax,Nx,dt,L):
    x = np.linspace(0, L, Nx)
    C = psi_R_anal(x,0) @ psi_R_anal(x,0) + psi_I_anal(x,0) @ psi_I_anal(x,0)
    psi_R_init = psi_R_anal(x, 0) / np.sqrt(C)
    psi_I_init = psi_I_anal(x, -dt/2) / np.sqrt(C)
    return x , psi_R_init, psi_I_init

In [ ]:
def timeEvolve(psi_R,psi_I,n,V,dt):
    Nx = len(V)
    dx = L / (Nx-1)
    
    D1 = V / hbar + hbar / (m * dx**2) * np.ones(Nx) 
    D2 = - hbar / (2 * m * dx**2) *  np.ones(Nx-1) 
    H = sp.diags([D2, D1 ,D2],[-1,0,1])
    for i in range(1,n):
        psi_I = psi_I - dt * H.dot(psi_R)
        psi_R = psi_R + dt * H.dot(psi_I)
    return psi_R,psi_I

        

In [ ]:
x0 = 5
sigmax = 1
Nx = 2000
V = np.zeros(Nx)
v_g = hbar * k0 / m
dt_2 = 0.0001
n_2 = int(L/ 2 * m / (hbar * k0 * dt_2))
    
x, psi_R_init,psi_I_init = make_psi_init(x0,sigmax,Nx,dt_2,L)
#x, psi_R_init, psi_I_init = make_psi_init(x0, sigmax, Nx, dt_2, L)
psi_R , psi_I = timeEvolve(psi_R_init,psi_I_init,n_2,V,dt_2)
plt.plot(x,psi_R)
plt.show()

In [ ]:
def plot_barrier(x, psi_init, psi_final, V):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(20, 8)
    
    ax1_2 = ax1.twinx()                           
    l1, = ax1.plot(x, V, color = "tab:red")
    l1_2, = ax1_2.plot(x, psi_init)                       
    
    ax2_2 = ax2.twinx()
    l2, = ax2.plot(x, V, color = "tab:red")                               
    l2_2, = ax2_2.plot(x, psi_final)                

In [ ]:
##Barrier, exercise 3
n = 100
L = 20
l = L / 50
b = E / 2.0
dt = 0.001

Vb = np.zeros(Nx)
Vk = np.ones(int(l / L * Nx)) * b
Vb[int(Nx/2 - l * Nx / L /  2): int(Nx/2 + l * Nx / L /  2) ] = Vk


x, psi_R_init, psi_I_init = make_psi_init(x0, sigmax, Nx, dt, L)

psi_R_b,psi_I_b = timeEvolve(psi_R_init, psi_I_init, n, V, dt)


plot_barrier(x, psi_R_init, psi_R_b, V)